In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow import keras as k
import random
from tqdm import tqdm
import shutil
import datetime, os

cwd = os.getcwd()
primary_dir = os.fsencode(cwd)



shutil.rmtree("./logs")
logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)

%reload_ext tensorboard
%tensorboard --logdir {logs_base_dir}

tensorboard_callback = k.callbacks.TensorBoard(log_dir=logs_base_dir)
writer = tf.summary.create_file_writer("./logs")

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/sebastian/anaconda3/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/home/sebastian/anaconda3/lib/python3.7/site-packages/tensorboard/main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "/home/sebastian/anaconda3/lib/python3.7/site-packages/absl/app.py", line 299, in run
    _run_main(main, args)
  File "/home/sebastian/anaconda3/lib/python3.7/site-packages/absl/app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "/home/sebastian/anaconda3/lib/python3.7/site-packages/tensorboard/program.py", line 289, in main
    return runner(self.flags) or 0
  File "/home/sebastian/anaconda3/lib/python3.7/site-packages/tensorboard/program.py", line 305, in _run_serve_subcommand
    server = self._make_server()
  File "/home/sebastian/anaconda3/lib/python3.7/site-packages/tensorboard/program.py",

In [2]:
binary_dict = {0: 'No triangle', 1 : 'Bias triangle'}

In [3]:
def create_model(pixel_size=32,dense_layers = [32*2,32],drop_out=0.4,l2=0.0001):
    model = models.Sequential()
    model.add(layers.Conv2D(pixel_size, (3, 3), activation='relu', input_shape=(pixel_size, pixel_size, 1), padding="SAME"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(pixel_size*2, (3, 3), activation='relu', padding="SAME"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(pixel_size*2, (3, 3), activation='relu', padding="SAME"))
    model.add(layers.Flatten())
    for dense_layer in dense_layers:
        model.add(layers.Dense(dense_layer, activation='elu',kernel_regularizer=regularizers.l2(l2)))
        model.add(layers.Dropout(drop_out))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

def compile_model(model):
    model.compile(optimizer='adam',
             loss='binary_crossentropy',
              metrics=['accuracy'])
    return model

def train_model(model,train_images,train_labels,test_images,test_labels,epochs,batch_size):
    history = model.fit(train_images,train_labels,shuffle=True, epochs=epochs, batch_size=batch_size, 
                        validation_data=(test_images,test_labels),callbacks=[tensorboard_callback])
    return model, history

def evaluate_model(model,history, test_images,test_labels):
    plt.clf()
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.0, 1.1])
    plt.legend(loc='lower right')
    plt.show()
    
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    return test_loss, test_acc

def normalise(x):
    x_max = np.amax(x)
    x_min = np.amin(x)
    x = (x - x_min)/(x_max-x_min)
    return x

In [4]:
data_total = np.load('data.npy')
labels_total = np.load('labels.npy')


data_total = normalise(data_total)

val_split = 0.3
val_set = np.int(val_split*np.shape(labels_total)[0])


train_images= np.array(data_total[val_set:]).reshape(-1, 32, 32, 1)
train_labels = np.array(labels_total[val_set:]).reshape(-1, 1)
validate_images = np.array(data_total[:val_set]).reshape(-1, 32, 32, 1)
validate_labels = np.array(labels_total[:val_set]).reshape(-1, 1)


def model_main(pixel_size=32,drop_out=0.0,l2=0.0001,epochs = 10,batch_size = 100):

    model_binary_classifier = create_model(pixel_size=pixel_size,drop_out=drop_out,l2=l2)
    model_binary_classifier.summary()
    model_binary_classifier = compile_model(model_binary_classifier)
    val_loss, val_acc = train_model(model_binary_classifier,train_images,train_labels,validate_images,validate_labels,epochs,batch_size)
    return model_binary_classifier

model_binary_classifier = model_main()

model_binary_classifier.save('bias_triangle_binary_classifier.h5') 